# Radar CAS 项目工作经验总结

## 所遇到的困难以及解决
### 管理
1. 由于项目比较小，没有像过去那样有项目经理，这一块经验不足，在一开始没有完全问清楚软件需求，得到的信息是来自三个不同的地方，每个人的说法都不一样，导致软件起手搭建时方向有点歪，浪费了不少时间。  之后应该更多挖掘并完全了解清楚软件需求并征询所有相关者的意见，意见不统一时需要大家协商。      

2. 项目无法按时完成容易心态爆炸，劳累容易心态爆炸。 老毛病了。不要老逼迫自己，做不完就做不完，大不了开了我，否则BUG越来越多。

## 技术
1. 在尝试画波图时，使用了pyqt自带的库qchart，拉跨得一匹，老是引起程序崩溃。由于使用python写多线程没有任何经验，所以找了一下午才找到是由于画图的线程崩溃引起的主线程崩溃。       


2. Python的线程解释器是带有全局锁的，之前虽然知道，但是没怎么接触过，这次是真正的接触到了。 所有的python线程都是无法真正并行的，即使是并发也是需要休眠才能进行切换。这个休眠多久就需要摸透，如果线程执行时间超过该线程休眠时间，任务会在内存中累积从而导致线程崩溃。    


3. 写灰度图代码时经过调研只能使用matplotlib并嵌入pyqt中，使用imshow函数无论如何也无法做到实时显示正确的图像，只做到直接加载历史数据并显示出来。 最后查看imshow的官方说明，看了好几遍，才明白，需要定义vmin和vmax的值，否则会按照初始化时候的vmin,vmax值计算导致实时数据回来时显示白色。其实还是不知道灰度图的计算方式导致的吧，灰度图需要有最大值和最小值才能正确计算出转换成GBR颜色的系数，否则数字大了，系数不正确就变成白色。    


4. 对接算法时，我自己的电脑无论如何也无法安装TENSORFLOW，一直提示cupy找不到，最后发现cupy是python调用GPU的函数库，GPU需要INVIDA显卡，而我的电脑，继承显卡。能装上就是想屁吃。 GPU是图像计算能力特别强，也就是说，它的设计是和CPU不用的，大概是精于计算矩阵结构的。    


5. 还是对接算法，换了一台TF环境配置已经搭建好的牛皮电脑。但是程序在使用本地数据模拟实时雷达时，读取数据频率为0.1，已经很慢了，但是在读到1万8数据就不可避免陷入崩溃，我一边写算法一边慢慢找，找了两三天，使用了当年在国台自己发明的一套检测系统性能的方式找到了元凶：numpy.append()函数。 我很少使用numpy的库，所以不了解库的函数，以为和python列表的append一样，万万没想到！这个函数的追加数据的机制的内存并不是动态分配的！！而是每追加新的元素，它会复制原有数据并开辟新的内存放进去。 这就导致了数据量收集越多，速度就越来越慢。 问题找到了就好修改了，把收集到的数据作为python的list元素储存在内存中，算法需要时候再截取对应的数据并转换成numpy。     


6. 加入GPS收集以后，有一个奇怪的现象，由于GPS回来的数据比较慢2Hz,而雷达可以有100Hz，于是该线程休眠时间相对比较长，可每次收了100多条GPS数据后，GPS连接会自动断掉。 我就很迷惑。如果不使用线程连续接收数据就不会产生这个问题。没办法，我又重新找了一些python线程的使用例子看，发现其实我的线程的写法有问题，这是PYQT的线程库，有信号槽，在线程类的run内执行的代码并不会影响线程执行时间，所以我应该把线程需要执行的主要内容放到run里，而不是主类调用的函数里。 这样线程执行的顺序变成：thread.start()==>run(==>collect data==>emit(data)）==>handling data， 而不是原先的：thread.start()==>run(==>emit())==>collect==>handling data, 这个顺序换了再也没出现过GPS断开连接的情况，甚至该线程再也没有因为时间问题而崩溃。 所以，我把雷达数据收集的线程也改成了这个模式, 能接收的数据量和速度从3.6W，20Hz变成了10w+， 100Hz。从坑里爬出来了。     


7. 在修改完GPS和雷达收数据的线程之后，程序速度的瓶颈到了计算这一块，虽然也可以把计算的线程改为GPS和雷达一样的模式，但是需要改动的地方太多甚至占用过多内存，在分析完至少20W+数据执行的性能报告里，很可能改了模式对于速度的提升也并不大，但如果不改，如果CPU或者GPU占用率高的时候会引起程序崩溃。 最后没改，至少目前在0.08秒的频率下从未出现过崩溃，10W数据可能会有些卡顿。 另外把计算prior这个过程给优化了不少，初始化TF时就把神经网络初始化一遍，避免首次计算特征时的卡顿，以及把剩下两个numpy.append()给灭了。而unregistered的过程依然还有numpy.append()函数，想来想去似乎没有好办法能避免使用这个函数。不过好在unregistered的计算并不密集，执行时间比较充裕，从未有过崩溃的情况。      


8. 整个项目代码层面的框架我还是照着Android项目抄的，最后发现似乎契合度并不高，毕竟它不需要多语言，图片素材什么的也不需要。对于qyqt我也不熟悉，学习时间过短，如果之前能明白findChild（）的机制，也许代码可以比现在更加简洁。